# Loop 1 Analysis: Understanding the Ceiling and Next Steps

## Objective
1. Analyze the baseline results
2. Understand what's limiting accuracy
3. Identify high-value improvements

In [1]:
import pandas as pd
import numpy as np

# Load data
train = pd.read_csv('/home/data/train.csv')
test = pd.read_csv('/home/data/test.csv')

print(f'Train: {train.shape}, Test: {test.shape}')
print(f'\nTarget distribution:')
print(train['Survived'].value_counts())

Train: (891, 12), Test: (418, 11)

Target distribution:
Survived
0    549
1    342
Name: count, dtype: int64


In [2]:
# Analyze the baseline feature importance from exp_000
# Top features were: Sex (0.45), Pclass (0.12), Fare (0.11), Age (0.08)

# Let's look at survival rates by key features
print('Survival by Sex:')
print(train.groupby('Sex')['Survived'].agg(['mean', 'count']))

print('\nSurvival by Pclass:')
print(train.groupby('Pclass')['Survived'].agg(['mean', 'count']))

print('\nSurvival by Sex and Pclass:')
print(train.groupby(['Sex', 'Pclass'])['Survived'].agg(['mean', 'count']))

Survival by Sex:
            mean  count
Sex                    
female  0.742038    314
male    0.188908    577

Survival by Pclass:
            mean  count
Pclass                 
1       0.629630    216
2       0.472826    184
3       0.242363    491

Survival by Sex and Pclass:
                   mean  count
Sex    Pclass                 
female 1       0.968085     94
       2       0.921053     76
       3       0.500000    144
male   1       0.368852    122
       2       0.157407    108
       3       0.135447    347


In [3]:
# Look at the hardest cases - where simple rules fail
# Female in 3rd class, Male in 1st class

print('Female 3rd class survival:', train[(train['Sex']=='female') & (train['Pclass']==3)]['Survived'].mean())
print('Male 1st class survival:', train[(train['Sex']=='male') & (train['Pclass']==1)]['Survived'].mean())

# These are the "noisy" cases where prediction is harder
print('\nThese groups have ~50% survival - essentially random')

Female 3rd class survival: 0.5
Male 1st class survival: 0.36885245901639346

These groups have ~50% survival - essentially random


In [4]:
# Analyze potential for improvement through ticket-based features
# Passengers with same ticket might be family/group

ticket_counts = train.groupby('Ticket').size()
print('Ticket group sizes:')
print(ticket_counts.value_counts().sort_index())

# Check if ticket groups have correlated survival
ticket_survival = train.groupby('Ticket')['Survived'].agg(['mean', 'count', 'std'])
ticket_survival = ticket_survival[ticket_survival['count'] > 1]
print(f'\nTickets with multiple passengers: {len(ticket_survival)}')
print(f'Mean survival rate variance within groups: {ticket_survival["std"].mean():.3f}')

Ticket group sizes:
1    547
2     94
3     21
4     11
5      2
6      3
7      3
Name: count, dtype: int64

Tickets with multiple passengers: 134
Mean survival rate variance within groups: 0.237


In [5]:
# Check cabin patterns more deeply
print('Cabin availability by Pclass:')
print(train.groupby('Pclass')['Cabin'].apply(lambda x: x.notna().mean()))

# Extract deck from cabin
train['Deck'] = train['Cabin'].str[0]
print('\nSurvival by Deck:')
print(train.groupby('Deck')['Survived'].agg(['mean', 'count']).sort_values('mean', ascending=False))

Cabin availability by Pclass:
Pclass
1    0.814815
2    0.086957
3    0.024440
Name: Cabin, dtype: float64

Survival by Deck:
          mean  count
Deck                 
D     0.757576     33
E     0.750000     32
B     0.744681     47
F     0.615385     13
C     0.593220     59
G     0.500000      4
A     0.466667     15
T     0.000000      1


In [6]:
# Analyze the theoretical ceiling
# If we had perfect predictions for the "easy" cases and 50% for "hard" cases

# Easy cases: Female (not 3rd class), Male 3rd class
easy_female = train[(train['Sex']=='female') & (train['Pclass']!=3)]
easy_male_3rd = train[(train['Sex']=='male') & (train['Pclass']==3)]

# Hard cases: Female 3rd class, Male 1st/2nd class
hard_female_3rd = train[(train['Sex']=='female') & (train['Pclass']==3)]
hard_male_upper = train[(train['Sex']=='male') & (train['Pclass']!=3)]

print('Easy cases (high confidence):')
print(f'  Female 1st/2nd class: {len(easy_female)} samples, {easy_female.Survived.mean():.1%} survival')
print(f'  Male 3rd class: {len(easy_male_3rd)} samples, {easy_male_3rd.Survived.mean():.1%} survival')

print('\nHard cases (lower confidence):')
print(f'  Female 3rd class: {len(hard_female_3rd)} samples, {hard_female_3rd.Survived.mean():.1%} survival')
print(f'  Male 1st/2nd class: {len(hard_male_upper)} samples, {hard_male_upper.Survived.mean():.1%} survival')

Easy cases (high confidence):
  Female 1st/2nd class: 170 samples, 94.7% survival
  Male 3rd class: 347 samples, 13.5% survival

Hard cases (lower confidence):
  Female 3rd class: 144 samples, 50.0% survival
  Male 1st/2nd class: 230 samples, 27.0% survival


In [7]:
# Calculate theoretical accuracy ceiling
# Assume perfect prediction on easy cases, 50% on hard cases

easy_correct = len(easy_female) + len(easy_male_3rd)  # Predict all correctly
hard_correct = (len(hard_female_3rd) + len(hard_male_upper)) * 0.5  # 50% correct

theoretical_ceiling = (easy_correct + hard_correct) / len(train)
print(f'Theoretical ceiling (simplified): {theoretical_ceiling:.1%}')

# More realistic: use actual survival rates as prediction confidence
from sklearn.metrics import accuracy_score

# Simple rule-based prediction
train['simple_pred'] = ((train['Sex']=='female') | (train['Pclass']==1)).astype(int)
simple_acc = accuracy_score(train['Survived'], train['simple_pred'])
print(f'Simple rule accuracy: {simple_acc:.1%}')

# Current baseline: 82.71% CV
print(f'Current baseline CV: 82.71%')
print(f'Gap to ceiling: ~{(0.84 - 0.8271)*100:.1f}% potential improvement')

Theoretical ceiling (simplified): 79.0%


Simple rule accuracy: 75.1%
Current baseline CV: 82.71%
Gap to ceiling: ~1.3% potential improvement
